In [ ]:
import numpy as np 
import pandas as pd 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Optimizer
from torch.autograd import Variable
import math
import cv2
from torchvision import transforms, models

import matplotlib.pyplot as plt
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
        #print(os.path.join(dirname, filename))
# we are using Kaggle kernel

In [ ]:
path = '../input/tiny-imagenet-challenge/TinyImageNet/'

In [ ]:
train_label = pd.read_csv(path+'train.txt', sep=' ', names=['image', 'label'])
test = pd.read_csv(path+'test.txt', sep=' ', names=['image'])
val_label = pd.read_csv(path+'val.txt', sep=' ', names=['image', 'label'])

print('train size: ', train_label.shape)
print('val size: ', val_label.shape)
print('test size: ', test.shape)

In [ ]:
class data(Dataset):
    def __init__(self, type, transform):
        self.type = type
        if type == 'train':
            self.train_images = []
            for row in train_label.itertuples(index=True, name='Pandas'):
                # print(getattr(row, 'image'))
                image_path = os.path.join(path, getattr(row, 'image'))
                self.train_images.append(cv2.imread(image_path))
            self.train_images = np.array(self.train_images)
            
        elif type == 'val':
            self.val_images = []
            for row in val_label.itertuples(index=True, name='Pandas'):
                # print(getattr(row, 'image'))
                image_path = os.path.join(path, getattr(row, 'image'))
                self.val_images.append(cv2.imread(image_path))
            self.val_images = np.array(self.val_images)
            
        self.transform = transform
        
    def __getitem__(self, index):
        label = []
        image = []
        if self.type == 'train':
            label = train_label.loc[index, 'label']
            image = self.train_images[index]
        if self.type == 'val':
            label = val_label.loc[index, 'label']
            image = self.val_images[index]
        return self.transform(image), label 
        
    def __len__(self):
        length = 0
        if self.type == 'train':
            length = self.train_images.shape[0]
        if self.type == 'val':
            length = self.val_images.shape[0]
        return length


In [ ]:
train_dataset = data('train', transform=transforms.ToTensor())
val_dataset = data('val', transform=transforms.ToTensor())

In [ ]:
batch_size = 128
val_batch_size = 128

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=val_batch_size, shuffle=False)


In [ ]:
class Mymodel(nn.Module):
    def __init__(self):
        super(Mymodel, self).__init__()
        self.resnet = model.resnet50() # pretrained model from torchvision
        self.linear1 = torch.nn.Linear(in_features=1000, out_features=512)
        self.linear2 = torch.nn.Linear(in_features=512, out_features=256)
        self.linear3 = torch.nn.Linear(in_features=256, out_features=100)
        
    def forward(self, x):
        x = self.se_resnet(x)
        x = F.relu(x)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        
        return x


In [ ]:
def build_model(path):
    #加载预训练参数
    resnet_state_dict = torch.load(path)
    model = Mymodel()
    model.load_state_dict(resnet_state_dict)
    return model

In [ ]:
mdoel = build_model(path='../input/resnet50/resnet50.pth')
print(model)

In [ ]:
max_epoch = 20

In [ ]:
use_gpu = torch.cuda.is_available()

In [ ]:
Train_Accuracy = []
Val_Accuracy = []
Train_Loss = []
Val_Loss = []

In [ ]:
if use_gpu:
    model = model.cuda()

criterion = nn.CrossEntropyLoss() 
# optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.95)
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=1e-5)


    
for epoch in range(max_epoch):
    train_acc_sum = 0
    train_sum = 0
    val_acc_sum = 0
    val_sum = 0
    train_Accuracy = []
    val_Accuracy = []
    train_Loss = []
    val_Loss = []

    model.train()
    print('***Training***')
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs
        inputs, labels = data
        if use_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()
            
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
        
        # Forward pass: Compute predicted y by passing X to the model
        y_pred = model(inputs)
        y_predict = y_pred.argmax(axis=1)
        
        train_acc_sum += float(sum(y_predict == labels))
        train_sum += float(len(labels))
        
        acc = train_acc_sum / float(train_sum)
        train_Accuracy.append(acc)
        
        # Compute and print Loss
        loss = criterion(y_pred, labels)
        if use_gpu: loss = loss.cpu()
        train_Loss.append(loss)
        
        if i % 100 == 0: 
            print('Epoch: {}\t Batch: {}\t Loss: {:.4f}\t Accuracy: {:.4f} '.format(epoch, i, loss, acc))
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    Train_Loss.append(float(torch.stack(train_Loss).mean()))
    Train_Accuracy.append(np.array(train_Accuracy).mean())
    
    torch.cuda.empty_cache()
    ################################
    with torch.no_grad():
        model.eval()
        print('***Validating***')
        for i, data in enumerate(val_dataloader, 0):
            # get the inputs
            inputs, labels = data
            if use_gpu:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # wrap them in Variable
            inputs, labels = Variable(inputs), Variable(labels)

            # Forward pass: Compute predicted y by passing X to the model
            y_pred = model(inputs)
            y_predict = y_pred.argmax(axis=1)

            val_acc_sum += float(sum(y_predict == labels))
            val_sum += float(len(labels))
            acc = val_acc_sum / float(val_sum)
            val_Accuracy.append(acc)

            loss = criterion(y_pred, labels)
            if use_gpu: loss = loss.cpu()
            val_Loss.append(loss)

            if i % 10 == 0: 
                print('Epoch: {}\t Batch: {}\t Loss: {:.4f}\t Accuracy: {:.4f} '.format(epoch, i, loss, acc))

        Val_Loss.append(float(torch.stack(val_Loss).mean()))
        Val_Accuracy.append(np.array(val_Accuracy).mean())

In [ ]:
torch.save(model.state_dict(),'./my_resnet50.pth')

In [ ]:
# plot loss curve and accuracy curve

In [ ]:
class test_data(Dataset):
    def __init__(self, transform):
        self.test_images = []
        self.id = []
        for row in test.itertuples(index=True, name='Pandas'):
            # print(getattr(row, 'image'))
            image_path = os.path.join(path, getattr(row, 'image'))
            self.test_images.append(cv2.imread(image_path))
            self.id.append(getattr(row, 'image').replace('test/', ''))
        self.test_images = np.array(self.test_images)
        self.id = np.array(self.id)
        self.transform = transform
        
    def __getitem__(self, index):
        label = self.id[index]
        image = self.test_images[index]
        return self.transform(image), label 
        
    def __len__(self):
        length = 0   
        length = self.test_images.shape[0]
        return length


In [ ]:
test_dataset = test_data(transform=transforms.ToTensor())
test_dataloader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

In [ ]:
Test_Sub = []
ID = []
with torch.no_grad():
    torch.cuda.empty_cache()
    model = model.cuda()
    for i, data in enumerate(test_dataloader, 0):
        # get the inputs
        inputs, ids = data
        if use_gpu:
            inputs = inputs.cuda()
            
        inputs = Variable(inputs)
        test_pred = model(inputs)
        test_sub = test_pred.argmax(axis=1)  
        test_sub = test_sub.cpu()
        Test_Sub = np.append(Test_Sub, test_sub)
        ID = np.append(ID, ids)

In [ ]:
my_submission = pd.DataFrame({'Id': ID, 'Category': Test_Sub})
my_submission.to_csv('./submission.csv', index=False)